In [0]:
from pyspark.sql import functions as F

tables = (
    spark.sql("SHOW TABLES IN skills_intelligence.01_bronze")
    .filter("tableName LIKE 'financial_sector_data_or_bi_analyst_%'")
    .collect()
)

if not tables:
    raise ValueError("Tidak ada table yang match pattern: financial_sector_data_or_bi_analyst_% di skills_intelligence.01_bronze")

dfs = []

for t in tables:
    table_name = t.tableName
    table_name_lc = table_name.lower()

    if "abnamro" in table_name_lc:
        company = "ABN AMRO"
    elif "ing" in table_name_lc:
        company = "ING"
    elif "devolksbank" in table_name_lc:
        company = "De Volksbank"
    else:
        # fallback: ambil token terakhir setelah prefix
        company = table_name.replace("financial_sector_data_or_bi_analyst_", "").upper()

    df = (
        spark.table(f"skills_intelligence.01_bronze.{table_name}")
        .withColumn("Company", F.lit(company))
        .withColumn("silver_loaded_at", F.current_timestamp())
        .withColumn("source_table", F.lit(table_name))
    )

    dfs.append(df)

final_df = dfs[0]
for df in dfs[1:]:
    final_df = final_df.unionByName(df, allowMissingColumns=True)

final_df.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable(
    "skills_intelligence.02_silver.financial_sector_data_or_bi_analyst"
)